# Distraction Driver Detection Project
## Project Aim: To predict the activity of the driver from the image
This is some plain text that forms a paragraph.
Add emphasis via **bold** and __bold__, or *italic* and _italic_.

Paragraphs must be separated by an empty line.

* Sometimes we want to include lists.
 * Which can be indented.

1. Lists can also be numbered.
2. For ordered lists.

[It is possible to include hyperlinks](https://www.example.com)

Inline code uses single backticks: `foo()`, and code blocks use triple backticks:

```
bar()
```

Or can be intented by 4 spaces:

    foo()


## Step 0: Importing the datasets

We start by importing the images from the datasets

In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
from glob import glob

#defining a function to load the dataset
def load_dataset(path):
    data = load_files(path)
    driver_images = np.array(data['filenames'])
    driver_activities = np_utils.to_categorical(np.array(data['target']))
    return driver_images, driver_activities

#loading the datasets
images, targets = load_dataset('imgs/train')

#splitting data to train, test and validation datasets
images_train, images_rest, targets_train, targets_rest = train_test_split( images, targets, train_size=0.8, random_state=42)
images_val, images_test, targets_val, targets_test = train_test_split( images_rest, targets_rest, train_size=0.5, random_state=42)


#printing the dataset statistics
print('There are %d total number of driver images' % len(images))



print('There are %d number of train images' % len(images_train))
print('There are %d number of validation images' % len(images_val))
print('There are %d number of test images' % len(images_test))



Using TensorFlow backend.


There are 22424 total number of driver images
There are 17939 number of train images
There are 2242 number of validation images
There are 2243 number of test images


## Step 2: Preprocessing

For preprocessing, we normalize the pixels in the images. That's more than enough.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tqdm import tqdm
#define a function that reads a path to an image and returns a tensor suitable for keras

def path_to_tensor(path):
    img = image.load_img(path, target_size=(128, 128))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)


#define a function that reads a path to an image and returns a tensor suitable for keras
def paths_to_tensor(paths_list):
    img_list = [path_to_tensor(img_path) for img_path in tqdm(paths_list)]
    return np.vstack(img_list)


from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

#normalize all images: divide the tensors by 255
# pre-process the data for Keras
train_tensors = paths_to_tensor(images_train).astype('float32')/255
valid_tensors = paths_to_tensor(images_val).astype('float32')/255
test_tensors = paths_to_tensor(images_test).astype('float32')/255




  6%|▋         | 1143/17939 [00:23<32:27,  8.63it/s]